In [6]:
import pandas as pd
import numpy as np
import json
import os
import re
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.pandas.cursor import PandasCursor

In [7]:
# Load client data
client = "volkswagen"
client_df = pd.read_excel(f"./{client}_input_data.xlsx", converters={"Imps": int, "Placement Id": int})
client_df["Viewability Rate %"] = client_df["Viewability Rate %"].str.replace(",", ".").astype(float)
client_placement_ids = list(client_df["Placement Id"].unique())

In [8]:
# Fetch impressions using PyAthena library
cursor = connect(s3_staging_dir="s3://aws-athena-query-results-094611745175-eu-west-1/",
                 region_name="eu-west-1", profile_name="atexprodadminsso", cursor_class=PandasCursor).cursor()

impressions_df = cursor.execute(f'''
select 
    "impression_model"."id", 
    "hostname", 
    "placement_ids", 
    "placement_ids_chosen", 
    "total_fixation_duration", 
    "channel", 
    "ad_technical_format", 
    "is_fixated", 
    "gaze_valid", 
    "is_iab_inview", 
    "exist_viewable_1_s_threshold_50", 
    "exist_viewable_2_s_threshold_50", 
    "impression_model"."part_month", 
    CONCAT(cast("ad_width_chosen" as VARCHAR), 'x', cast("ad_width_chosen" as VARCHAR)) as size,
    "brand_batch"."chosen_brand" as "chosen_brand"
from "prod_attentionpanel_com_eu_west_1"."impression_model"
join "prod_attentionpanel_com_eu_west_1"."brand_batch" on impression_model.id = brand_batch.id
where impression_model.part_year = '2023'
''').as_pandas()

In [9]:
def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    try:
        ret = placement_id['tag_id'][0]
    except:
        return
    ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/tmp/ipykernel_8567/3375560408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))


In [10]:
# Get local predictions and store them in a map
def _get_pred_map():
    pred_map = dict()
    _year = 2023
    _range1 = 1
    _range2 = 11
    for i in range(_range1, _range2):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../../predictions/{_year}/{i}'):
            for h in os.listdir(f'../../predictions/{_year}/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../../predictions/{_year}/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [50]:
# Filter impressions based on tier
### TIER 1 ###
# This is the most accurate data we have. This is "campaign data", meaning correct brand and timeframe
### TIER 2 ###
# We might not have enough campaign data (tier 1). Tier 2 data is brand data, for a longer period (roughly 6 months)
### TIER 3 ###
# This is the least accurate data for a campaign. It ignore brand/timeframe and only checks Placement ID. This is mostly used as a benchmark
tier = 1
filtered_impressions = impressions_df #impressions_df[impressions_df["pid"].isin(client_placement_ids)]
if tier == 1:
    filtered_impressions = filtered_impressions[filtered_impressions["part_month"] == "10"]
if tier < 3:
    filtered_impressions = filtered_impressions[filtered_impressions["chosen_brand"] == client.lower()]

In [51]:
filtered_impressions

,id,hostname,placement_ids,placement_ids_chosen,total_fixation_duration,channel,ad_technical_format,is_fixated,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,part_month,size,chosen_brand,pid,pid_type


In [28]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

filtered_impressions['final_fixation'] = filtered_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

/tmp/ipykernel_8567/1232414527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions['final_fixation'] = filtered_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)


In [29]:
filtered_impressions["true_is_iab_inview"] = filtered_impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

/tmp/ipykernel_8567/2412417818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions["true_is_iab_inview"] = filtered_impressions.apply(


In [30]:
filtered_impressions["total_fixation_duration"] = filtered_impressions["total_fixation_duration"].astype("Int64")
# Normalise outliers for fixation duration (30 seconds)
filtered_impressions.loc[filtered_impressions["total_fixation_duration"] >= 30000, "total_fixation_duration"] = 30000

/tmp/ipykernel_8567/3020262113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions["total_fixation_duration"] = filtered_impressions["total_fixation_duration"].astype("Int64")


In [31]:
grouped_df = filtered_impressions.groupby(["pid", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "impressions": x["id"].count(),
            "fixations": x.loc[x["final_fixation"], "id"].count(),
            "inview": x.loc[x["true_is_iab_inview"], "id"].count(),
            "total_fixation_duration": x["total_fixation_duration"].sum(),
            "average_fixation_duration": x["total_fixation_duration"].mean()
        }
    )
)
grouped_df = grouped_df.reset_index()

In [32]:
grouped_client = client_df.groupby(["Placement Id"]).apply(
    lambda x: pd.Series(
        {
            "Imps": x["Imps"].sum(),
            "Viewability Rate %": np.average(x["Viewability Rate %"], weights=x["Imps"])
        }
    )
)
grouped_client = grouped_client.reset_index()

In [33]:
merged_df = grouped_df.merge(grouped_client, how="inner", left_on="pid", right_on="Placement Id")
merged_df = merged_df[["hostname", "Placement Id", "impressions", "fixations", "inview", "Imps", "Viewability Rate %", "average_fixation_duration", "total_fixation_duration"]]
merged_df["fixation_ratio"] = round(merged_df["fixations"] / merged_df["impressions"], 2)
merged_df["client_fixations_per_placement"] = (merged_df["fixation_ratio"] * merged_df["Imps"]).fillna(0).replace([np.inf, -np.inf], 0)

In [34]:
final_group = merged_df.groupby(["hostname"]).apply(
     lambda x: pd.Series(
         {
             "tobii_imps": x["impressions"].sum(),
             "client_imps": x["Imps"].sum(),
             "tobii_fixations": x["fixations"].sum(),
             "tobii_inview": x["inview"].sum(),
             "client_fixations_per_placement": x["client_fixations_per_placement"].sum(),
             "client_viewability_rate": np.average(x["Viewability Rate %"], weights=x["Imps"]),
             "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 0),
             "total_fixation_duration": x["total_fixation_duration"].sum()
         }
     )
)
final_group = final_group.reset_index()
final_group["tobii_imps"] = final_group["tobii_imps"].astype("Int64")
final_group["client_imps"] = final_group["client_imps"].astype("Int64")
final_group["tobii_fixations"] = final_group["tobii_fixations"].astype("Int64")
final_group["tobii_inview"] = final_group["tobii_inview"].astype("Int64")
final_group["tobii_inview/impression_ratio"] = round(final_group["tobii_inview"] / final_group["tobii_imps"] * 100, 2)
final_group["tobii_fixation/impression_ratio"] = round(final_group["tobii_fixations"] / final_group["tobii_imps"] * 100, 2)
final_group = final_group.fillna(0).replace([np.inf, -np.inf], 0)

/tmp/ipykernel_8567/2389419411.py:10: RuntimeWarning: invalid value encountered in scalar divide
  "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 0),


In [35]:
final_group["client_inview"] = round(final_group["client_imps"] * final_group["client_viewability_rate"], 0).astype("Int64")
final_group["client_fixations"] = round(final_group["client_fixations_per_placement"], 0).astype("Int64")
final_group["client_fix/inview_ratio"] = round(final_group["client_fixations"] / final_group["client_inview"] * 100, 2)
final_group["client_total_fixation_duration"] = final_group["client_fixations"] * final_group["average_fixation_duration"]

def get_sample_size(num_impressions):
    if num_impressions <= 99:
        return "low"
    elif num_impressions >= 100 and num_impressions <= 199:
        return "medium"
    else:
        return "high"

final_group["sample_size"] = final_group["tobii_imps"].apply(lambda x: get_sample_size(x))

In [36]:
final_group

,hostname,tobii_imps,client_imps,tobii_fixations,tobii_inview,client_fixations_per_placement,client_viewability_rate,average_fixation_duration,total_fixation_duration,tobii_inview/impression_ratio,tobii_fixation/impression_ratio,client_inview,client_fixations,client_fix/inview_ratio,client_total_fixation_duration,sample_size
0,aftonbladet.se,109053,2355765,30323,66517,893694.49,0.742023,2115.0,64139453.0,61.0,27.81,1748033,893694,51.13,1890162810.0,high
1,bloggar.aftonbladet.se,273,97415,152,148,61781.47,0.695250,3237.0,491980.0,54.21,55.68,67728,61781,91.22,199985097.0,high
2,klart.se,1328,15944,402,738,5058.22,0.607905,1087.0,436871.0,55.57,30.27,9692,5058,52.19,5498046.0,high
3,malservice.aftonbladet.se,67,25,13,20,4.75,0.680000,271.0,3523.0,29.85,19.4,17,5,29.41,1355.0,low
4,skonhetsredaktorerna.se,2,39382,2,2,39382.00,0.703884,0.0,0.0,100.0,100.0,27720,39382,142.07,0.0,low
5,svd.se,10662,37285,5490,6895,14431.57,0.808458,7338.0,40287453.0,64.67,51.49,30143,14432,47.88,105902016.0,high
6,tramsfrans.aftonbladet.se,2,39382,0,2,0.00,0.703884,0.0,0.0,100.0,0.0,27720,0,0.0,0.0,low
7,tv.nu,853,71485,228,504,17362.71,0.694913,1766.0,402596.0,59.09,26.73,49676,17363,34.95,30663058.0,high


In [43]:
result_df = final_group[["hostname", "tobii_imps", "client_imps", "tobii_fixations", "client_fixations", "tobii_inview", "client_inview", "client_fix/inview_ratio", "tobii_inview/impression_ratio", "client_viewability_rate", "tobii_fixation/impression_ratio", "average_fixation_duration", "total_fixation_duration", "client_total_fixation_duration", "sample_size"]]
result_df
result_df.to_excel(f"./{client.lower()}_tier_{tier}_example_report.xlsx", index=False)

In [44]:
# Load tiered data
#df_1 = pd.read_excel(f"./{client.lower()}_tier_1_example_report.xlsx")
df_1 = pd.DataFrame({})
df_1["tier"] = 1
df_2 = pd.read_excel(f"./{client.lower()}_tier_2_example_report.xlsx")
df_2["tier"] = 2
df_3 = pd.read_excel(f"./{client.lower()}_tier_3_example_report.xlsx")
df_3["tier"] = 3
all_df = [df_1, df_2, df_3]
combined_client_df = pd.concat(all_df)
columns = list(df_2.columns)

In [45]:
def get_data_based_on_tier(client_dfs):
    new_df = pd.DataFrame(columns=columns)
    for i, df in enumerate(client_dfs):
        for index, row in df.iterrows():
            if row.tier == 1:
                if row.sample_size in ("high", "medium"):
                    new_df = new_df.append(row)
            elif row.tier == 2:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0 and row.tobii_imps >= 10:
                    if row.sample_size == "high":
                        # Degrade sample_size if lower tier
                        row.sample_size = "medium"
                        new_df = new_df.append(row)
                    elif row.sample_size == "medium":
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            round(row["client_fixations"] * 0.25 + new_row["client_fixations"].sum() * 0.75, 0)
                        )
                        new_df = new_df.append(new_row)
                    else:
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            round(row["client_fixations"] * 0.15 + new_row["client_fixations"].sum() * 0.85, 0)
                        )
                        new_df = new_df.append(new_row)
            elif row.tier == 3:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    row["sample_size"] = "low"
                    new_df = new_df.append(row)
    return new_df

final_client_df = get_data_based_on_tier(all_df)

/tmp/ipykernel_8567/764905171.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
/tmp/ipykernel_8567/764905171.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.sample_size = "medium"
/tmp/ipykernel_8567/764905171.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row["client_fixations"] = (
/tmp/ipykernel_8567/764905171.py:27: FutureWarning: The frame.append method is depr

In [46]:
final_client_df

,hostname,tobii_imps,client_imps,tobii_fixations,client_fixations,tobii_inview,client_inview,client_fix/inview_ratio,tobii_inview/impression_ratio,client_viewability_rate,tobii_fixation/impression_ratio,average_fixation_duration,total_fixation_duration,client_total_fixation_duration,sample_size,tier
0,aftonbladet.se,1048,1552689,386,593465,655,1160097,51.16,62.50,0.747154,36.83,4479,1728974,2658129735,medium,2
2,klart.se,1328.0,15944.0,402.0,5138.0,738.0,9692.0,52.19,55.57,0.607905,30.27,1087.0,436871.0,5498046.0,medium,3
5,svd.se,10662.0,37285.0,5490.0,13542.0,6895.0,30143.0,47.88,64.67,0.808458,51.49,7338.0,40287453.0,105902016.0,medium,3
7,tv.nu,853.0,71485.0,228.0,16506.0,504.0,49676.0,34.95,59.09,0.694913,26.73,1766.0,402596.0,30663058.0,medium,3
1,bloggar.aftonbladet.se,273,97415,152,61781,148,67728,91.22,54.21,0.695250,55.68,3237,491980,199985097,low,3
3,malservice.aftonbladet.se,67,25,13,5,20,17,29.41,29.85,0.680000,19.40,271,3523,1355,low,3
4,skonhetsredaktorerna.se,2,39382,2,39382,2,27720,142.07,100.00,0.703884,100.00,0,0,0,low,3
6,tramsfrans.aftonbladet.se,2,39382,0,0,2,27720,0.00,100.00,0.703884,0.00,0,0,0,low,3


In [47]:
def do_math(row, arg1, arg2):
    try:
        return round(row[f"{arg1}"] / row[f"{arg2}"] * 100, 2)
    except ZeroDivisionError:
        return 0

final_client_df["client_Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_inview", "client_imps"), axis=1)
final_client_df["client_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_client_df["tobii_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_client_df["client_fixation/impression_ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_client_df["Total Fixation Duration"] = final_client_df["average_fixation_duration"] * final_client_df["client_fixations"]
final_client_df = final_client_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "Total Fixation Duration", "average_fixation_duration", "sample_size", "tier"]]
final_client_df = final_client_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        "client_Inview Ratio": "Inview Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "average_fixation_duration": "Average Fixation Duration",
        "sample_size": "Sample Size"
    }
)
final_client_df

,hostname,Impressions,Fixations,tobii_fixation/impression_ratio,Inviews,Inview Ratio,Fixation/Inview Ratio,Fixation/Tobii Inview Ratio,tobii_inview/impression_ratio,Total Fixation Duration,Average Fixation Duration,Sample Size,tier
0,aftonbladet.se,1552689,593465,36.83,1160097,74.72,51.16,58.93,62.50,2658129735,4479,medium,2
2,klart.se,15944.0,5138.0,30.27,9692.0,60.79,53.01,54.47,55.57,5585006.0,1087.0,medium,3
5,svd.se,37285.0,13542.0,51.49,30143.0,80.84,44.93,79.62,64.67,99371196.0,7338.0,medium,3
7,tv.nu,71485.0,16506.0,26.73,49676.0,69.49,33.23,45.24,59.09,29149596.0,1766.0,medium,3
1,bloggar.aftonbladet.se,97415,61781,55.68,67728,69.53,91.22,102.70,54.21,199985097,3237,low,3
3,malservice.aftonbladet.se,25,5,19.40,17,68.00,29.41,65.00,29.85,1355,271,low,3
4,skonhetsredaktorerna.se,39382,39382,100.00,27720,70.39,142.07,100.00,100.00,0,0,low,3
6,tramsfrans.aftonbladet.se,39382,0,0.00,27720,70.39,0.00,0.00,100.00,0,0,low,3


In [48]:
final_client_df = final_client_df[["hostname", "Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration", "Sample Size"]]
final_client_df = final_client_df.sort_values(by=["hostname"])

In [49]:
comments = ["Tier 1", "Tier 2", "Tier 3"]

with pd.ExcelWriter(f"./{client}_results_internal.xlsx") as writer:
    rows = 0
    spaces = 0
    for n, df in enumerate(all_df):
        pd.Series(comments[n]).to_excel(writer, sheet_name=client.title(), index=False, header=False, startrow=rows + spaces)
        df.to_excel(writer, client.title(), index=False, startrow=1 + rows + spaces)
        rows += len(df) + 2
        spaces += 2

with pd.ExcelWriter(f"./{client}_results.xlsx") as writer:
    for df in [final_client_df]:
        df.to_excel(writer, sheet_name=client.title(), index=False)